## Overview ##

PubChem is a site run by the NIH which hosts raw data associated with chemical experiments; here we analyze the data hosted at PubChem for assay 1030, which looks for inhibitors of the protein encoding gene ALDH1A1. You can access the page for this assay [here](https://pubchem.ncbi.nlm.nih.gov/bioassay/1030)

## Results ##

We use the SMILES string, a common representation for a molecule amongst chemists, to begin the featurization process. Because the length of this string varies, it is normalized in the form of a Morgan Fingerprint; these are then used to train a RandomForestClassifier using sklearn. The performance on a held-out test set was approx 93% 


In [14]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

global_random_state = 42

np.random.seed(global_random_state)

In [ ]:
# Load assay info. Note: This CSV was obtained from PubChem bioassay aka PCBA, via searching for AID 1030 
# and downloading the datatable

ba_df = pd.read_csv("AID_1030_datatable_all.csv")

# Load compound info
cs_df = pd.read_csv("AID_1030_compound_smiles.csv",sep='\t',header=0)

# Merge the two
full_df = ba_df.merge(cs_df,on='PUBCHEM_CID')

# Cleanup the compound ID column
full_df["PUBCHEM_CID"] = full_df["PUBCHEM_CID"].astype(int)

compound_ids = list()
fingerprints = list()
activities = list()
smiles_list = list()

for index, row in full_df.iterrows() :
    cid = row["PUBCHEM_CID"]
    smiles_string = row["Smiles"]
    mol = Chem.MolFromSmiles(smiles_string)
    is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
    if mol is not None: 
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=2048,useChirality=False,
                                                                     useBondTypes=False,useFeatures=False)
        
        # From RDKit documentation
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        fingerprint = arr
        
        compound_ids.append(cid)
        smiles_list.append(smiles_string)
        fingerprints.append(fingerprint)
        activities.append(is_active)
    
    if index % 10000 == 0:
        print("Processed index: {0}".format(index))

print("Processed all, pickling")

compound_ids_and_features = (compound_ids, smiles_list, fingerprints, activities)

# Pickle the data to save time in the future
with open('data.pickle', 'wb') as f:
    pickle.dump(compound_ids_and_features, f, pickle.HIGHEST_PROTOCOL)

Processed index: 0
Processed index: 10000
Processed index: 20000
Processed index: 30000
Processed index: 40000
Processed index: 50000
Processed index: 60000
Processed index: 70000
Processed index: 80000
Processed index: 90000
Processed index: 100000
Processed index: 110000
Processed index: 120000
Processed index: 130000
Processed index: 140000
Processed index: 150000
Processed index: 160000
Processed index: 170000
Processed index: 180000
Processed index: 190000
Processed index: 200000
Processed index: 210000
Processed index: 220000
Processed all, pickling


In [1]:
# Duplicate imports in case starting from this cell

import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors

compound_ids = None
fingerprints = None
activities = None

with open('data.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (compound_ids, smiles_list, fingerprints, activities) = pickle.load(f)

# Print the number of compounds loaded
print("Successfully featurized {0} compounds.".format(len(compound_ids)))

# Show an example of the raw data
print("Sample ID: {0}".format(compound_ids[0]))
print("Sample fingerprint vector: ".format(fingerprints[0]))
print("Was it bioactive? Assay returned: {0}".format(activities[0]))

Successfully featurized 220364 compounds.
Sample ID: 6603008
Sample fingerprint vector: 
Was it bioactive? Assay returned: False


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# Create a train/test split
X_train, X_test, y_train, y_test = train_test_split(fingerprints, activities, test_size=0.33, random_state=global_random_state)

classifier = DecisionTreeClassifier(random_state=42)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

# Note: Unfortunately it's not directly comparable to ROC_AUC calculated in MoleculeNet at: https://arxiv.org/pdf/1703.00564.pdf 
# This is because MoleculeNet looks at a different metric (roc_auc) and also a different task (multiclass prediction across 128 bioassays simultaneously vs binary classification here)

Classifier obtained a mean accuracy score of: 0.9003726571416785
Precision: [ 0.94324277  0.30084484], Recall: [ 0.94966547  0.2748494 ], F-Score: [ 0.94644322  0.28726021], Support: [67409  5312]


In [7]:
# Let's try undersampling the majority class to improve the f1-score for True predictions
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_sample(fingerprints, activities)
print(sorted(Counter(y_resampled).items()))

[(False, 16111), (True, 16111)]


In [9]:
from sklearn.metrics import classification_report


# Create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.33, random_state=global_random_state)

classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

classifier = DecisionTreeClassifier(random_state=42)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))


Classifier obtained a mean accuracy score of: 0.6444423547113034
Precision: [ 0.6437959   0.64505495], Recall: [ 0.63218827  0.65647717], F-Score: [ 0.63793929  0.65071594], Support: [5269 5365]


In [11]:
# Let's try using a Random forest

classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))


Classifier obtained a mean accuracy score of: 0.7187323678766222
             precision    recall  f1-score   support

      False       0.70      0.76      0.73      5269
       True       0.74      0.68      0.71      5365

avg / total       0.72      0.72      0.72     10634



In [13]:
# Does an MLP classifier help?
# Answer: Nope, not really

from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(random_state=42)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))



Classifier obtained a mean accuracy score of: 0.7008651495204062
             precision    recall  f1-score   support

      False       0.70      0.70      0.70      5269
       True       0.70      0.71      0.70      5365

avg / total       0.70      0.70      0.70     10634



In [20]:
# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_dim=2048, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

y_train_binary = y_train.astype(int)
y_test_binary = y_test.astype(int)

model.fit(X_train, y_train_binary, epochs=150, batch_size=10)

scores = model.evaluate(X_test, y_test_binary)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/150
21588/21588 [==============================] - 2s - loss: 0.6285 - acc: 0.6461     
Epoch 2/150
21588/21588 [==============================] - 1s - loss: 0.5654 - acc: 0.7083     
Epoch 3/150
21588/21588 [==============================] - 1s - loss: 0.5204 - acc: 0.7446     
Epoch 4/150
21588/21588 [==============================] - 1s - loss: 0.4709 - acc: 0.7759     
Epoch 5/150
21588/21588 [==============================] - 1s - loss: 0.4246 - acc: 0.8077     
Epoch 6/150
21588/21588 [==============================] - 1s - loss: 0.3797 - acc: 0.8306     
Epoch 7/150
21588/21588 [==============================] - 2s - loss: 0.3400 - acc: 0.8545     
Epoch 8/150
21588/21588 [==============================] - 2s - loss: 0.3039 - acc: 0.8717     
Epoch 9/150
21588/21588 [==============================] - 1s - loss: 0.2725 - acc: 0.8854     
Epoch 10/150
21588/21588 [==============================] - 2s - loss: 0.2441 - acc: 0.9001     
Epoch 11/150
21588/21588 [=============

21588/21588 [==============================] - 1s - loss: 0.0131 - acc: 0.9963     
Epoch 86/150
21588/21588 [==============================] - 1s - loss: 0.0146 - acc: 0.9952     
Epoch 87/150
21588/21588 [==============================] - 1s - loss: 0.0119 - acc: 0.9965     
Epoch 88/150
21588/21588 [==============================] - 1s - loss: 0.0098 - acc: 0.9973     
Epoch 89/150
21588/21588 [==============================] - 1s - loss: 0.0097 - acc: 0.9972     
Epoch 90/150
21588/21588 [==============================] - 1s - loss: 0.0131 - acc: 0.9962     
Epoch 91/150
21588/21588 [==============================] - 1s - loss: 0.0118 - acc: 0.9961     
Epoch 92/150
21588/21588 [==============================] - 1s - loss: 0.0111 - acc: 0.9963     
Epoch 93/150
21588/21588 [==============================] - 1s - loss: 0.0101 - acc: 0.9971     
Epoch 94/150
21588/21588 [==============================] - 1s - loss: 0.0106 - acc: 0.9969     
Epoch 95/150
21588/21588 [=================

In [29]:
y_pred = model.predict(X_test)
#y_pred_classified = np.argmax(y_pred, axis=1)

print(y_pred[0:5])

y_pred_classified = np.argmax(y_pred, axis=1)

print(y_pred_classified[0:5])

y_pred_binarized = [x > .5 for x in y_pred_classified]

print(y_pred_binarized[0:5])

print(classification_report(y_test, y_pred_binarized))


[[  2.03750189e-03]
 [  1.14329438e-17]
 [  9.91880357e-01]
 [  1.11250974e-01]
 [  1.94710310e-32]]
[0 0 0 0 0]
[False, False, False, False, False]
             precision    recall  f1-score   support

      False       0.50      1.00      0.66      5269
       True       0.00      0.00      0.00      5365

avg / total       0.25      0.50      0.33     10634

